<a href="https://colab.research.google.com/github/saks0106/DeepLearning_NLP/blob/main/4_Translation_Bahdanau_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import sklearn### machine learning library
import cv2## image processing
from sklearn.metrics import confusion_matrix, roc_curve### metrics
import seaborn as sns### visualizations
import datetime
import pathlib
import io
import os
import re
import string
import time
from numpy import random
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import (Dense,Flatten,SimpleRNN,InputLayer,Conv1D,Bidirectional,GRU,LSTM,BatchNormalization,Dropout,Input, Embedding,TextVectorization)
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from google.colab import files
from tensorboard.plugins import projector

In [2]:
#In the previous books, we had problem of Long term memory ie info from inital blocks gets lost in the final steps and model performs poorly
# New problem - take a look around you pad pad pad --> regardez autour de vous [end], here entire input is sent to french block without any connection
  # regardez is connected to look but it has no way of knowing it, same for autour and around
# We focus on block level connection in input and output and how much every input block affected its every output block ie
  # how every block in input affects every block in the output. We use Attention Mechansim

# Data Preparation

## Data Download

In [3]:
!wget https://www.manythings.org/anki/fra-eng.zip

--2023-07-01 12:59:22--  https://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7420323 (7.1M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.08M  20.2MB/s    in 0.4s    

2023-07-01 12:59:23 (20.2 MB/s) - ‘fra-eng.zip’ saved [7420323/7420323]



In [4]:
!unzip "/content/fra-eng.zip" -d "/content/dataset/"

Archive:  /content/fra-eng.zip
  inflating: /content/dataset/_about.txt  
  inflating: /content/dataset/fra.txt  


## Kaggle Dataset

In [5]:
# !pip install -q kaggle
# !mkdir ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 /root/.kaggle/kaggle.json
# !kaggle datasets download -d dhruvildave/en-fr-translation-dataset

In [6]:
# !unzip "/content/en-fr-translation-dataset.zip" -d "/content/dataset/"

In [7]:
# dataset = tf.data.experimental.CsvDataset(
#   "/content/dataset/en-fr.csv",
#   [
#     tf.string,
#     tf.string
#   ],
# )

## Data Processing

In [8]:
text_dataset=tf.data.TextLineDataset("/content/dataset/fra.txt")

In [9]:
VOCAB_SIZE=20000
ENGLISH_SEQUENCE_LENGTH=64
FRENCH_SEQUENCE_LENGTH=64
EMBEDDING_DIM=300
BATCH_SIZE=64

In [10]:
english_vectorize_layer=TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=ENGLISH_SEQUENCE_LENGTH
)

In [11]:
french_vectorize_layer=TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=FRENCH_SEQUENCE_LENGTH
)

In [12]:
def selector(input_text):
  split_text=tf.strings.split(input_text,'\t')
  return {'input_1':split_text[0:1],'input_2':'starttoken '+split_text[1:2]},split_text[1:2]+' endtoken'

In [13]:
split_dataset=text_dataset.map(selector)

In [14]:
for i in split_dataset.take(3):
  print(i)

({'input_1': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, 'input_2': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'starttoken Va !'], dtype=object)>}, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Va ! endtoken'], dtype=object)>)
({'input_1': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, 'input_2': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'starttoken Marche.'], dtype=object)>}, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Marche. endtoken'], dtype=object)>)
({'input_1': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, 'input_2': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'starttoken En route !'], dtype=object)>}, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'En route ! endtoken'], dtype=object)>)


In [15]:
def separator(input_text):
  split_text=tf.strings.split(input_text,'\t')
  return split_text[0:1],'starttoken '+split_text[1:2]+' endtoken'

In [16]:
init_dataset=text_dataset.map(separator)

In [17]:
for i in init_dataset.take(3):
  print(i)

(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'starttoken Va ! endtoken'], dtype=object)>)
(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'starttoken Marche. endtoken'], dtype=object)>)
(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'starttoken En route ! endtoken'], dtype=object)>)


In [18]:
english_training_data=init_dataset.map(lambda x,y:x)### input x,y and output x
english_vectorize_layer.adapt(english_training_data)#### adapt the vectorize_layer to the training data

In [19]:
french_training_data=init_dataset.map(lambda x,y:y)### input x,y,z and output y
french_vectorize_layer.adapt(french_training_data)#### adapt the vectorize_layer to the training data

In [20]:
def vectorizer(inputs,output):
  return {'input_1':english_vectorize_layer(inputs['input_1']),
          'input_2':french_vectorize_layer(inputs['input_2'])},french_vectorize_layer(output)

In [21]:
split_dataset

<_MapDataset element_spec=({'input_1': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'input_2': TensorSpec(shape=(None,), dtype=tf.string, name=None)}, TensorSpec(shape=(None,), dtype=tf.string, name=None))>

In [22]:
dataset=split_dataset.map(vectorizer)

In [23]:
for i in split_dataset.take(3):
  print(i)

({'input_1': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, 'input_2': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'starttoken Va !'], dtype=object)>}, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Va ! endtoken'], dtype=object)>)
({'input_1': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, 'input_2': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'starttoken Marche.'], dtype=object)>}, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Marche. endtoken'], dtype=object)>)
({'input_1': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, 'input_2': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'starttoken En route !'], dtype=object)>}, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'En route ! endtoken'], dtype=object)>)


In [24]:
for i in dataset.take(1):
  print(i)

({'input_1': <tf.Tensor: shape=(1, 64), dtype=int64, numpy=
array([[44,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])>, 'input_2': <tf.Tensor: shape=(1, 64), dtype=int64, numpy=
array([[  2, 104,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])>}, <tf.Tensor: shape=(1, 64), dtype=int64, numpy=
array([[104,   3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,

In [25]:
dataset

<_MapDataset element_spec=({'input_1': TensorSpec(shape=(None, 64), dtype=tf.int64, name=None), 'input_2': TensorSpec(shape=(None, 64), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, 64), dtype=tf.int64, name=None))>

In [26]:
dataset=dataset.shuffle(2048).unbatch().batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

In [27]:
dataset

<_PrefetchDataset element_spec=({'input_1': TensorSpec(shape=(None, 64), dtype=tf.int64, name=None), 'input_2': TensorSpec(shape=(None, 64), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, 64), dtype=tf.int64, name=None))>

In [28]:
NUM_BATCHES=int(200000/BATCH_SIZE)

In [29]:
train_dataset=dataset.take(int(0.9*NUM_BATCHES))
val_dataset=dataset.skip(int(0.9*NUM_BATCHES))

In [30]:
train_dataset

<_TakeDataset element_spec=({'input_1': TensorSpec(shape=(None, 64), dtype=tf.int64, name=None), 'input_2': TensorSpec(shape=(None, 64), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, 64), dtype=tf.int64, name=None))>

# Modeling

## Encoder

In [31]:
# VOCAB_SIZE=20000
# ENGLISH_SEQUENCE_LENGTH=64
# FRENCH_SEQUENCE_LENGTH=64
# EMBEDDING_DIM=300
# BATCH_SIZE=64

In [32]:
class Encoder(tf.keras.Model):
  def __init__(self,vocab_size,embedding_dim,units): #20000,256,
    super(Encoder,self).__init__()
    self.embedding_dim=embedding_dim
    self.vocab_size=vocab_size
    self.units=units

  def build(self,input_shape): #building the input skeleton block
    self.embedding=Embedding(self.vocab_size,self.embedding_dim)
    self.lstm=LSTM(self.units,return_sequences=True) # play with using Bi-directioanl, GRU, return_sequences will give output from all blocks

  def call(self,x): # input passed in embedding and making weights ready to be passed
    x=self.embedding(x)
    output=self.lstm(x)
    return output

In [51]:
HIDDEN_UNITS=127 #for lstm
EMBEDDING_DIM=256  #for embedding
encoder=Encoder(VOCAB_SIZE,EMBEDDING_DIM,HIDDEN_UNITS) #20000,256,127 #build
encoder_output=encoder(tf.zeros([128,8])) #this is input (batch, sequence length)-> no of blocks each having a dims of 256 #cal;
print(encoder_output.shape)

#till now every weight has dims of 256 from every block. Now this will be passed through a fully connected layer and reduce dims to 32
#we have batch of 128, blocks of 8 and each block dims is 127 and every block will produce 127 dims

(128, 8, 127)


## Bahdanau Attention

Output from Above will be Received in Attention

In [38]:
tf.expand_dims([1,2,3,4,5,5],axis=1)

<tf.Tensor: shape=(6, 1), dtype=int32, numpy=
array([[1],
       [2],
       [3],
       [4],
       [5],
       [5]], dtype=int32)>

In [52]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self,units): #256
    super(BahdanauAttention,self).__init__()
    self.units=units #256

  def build(self,input_shape): # 256 dims from prev step is passed into dense layer to learn patterns
    """
    Creating Hiddens layers
    """
    self.w_1=tf.keras.layers.Dense(self.units) #256,  from decoder having shape (batch,32) comes into encoder pass through hidden layer of 256 units
    self.w_2=tf.keras.layers.Dense(self.units) #256,    127 dims from above will pass through FCL of 256--> 32
    self.w=tf.keras.layers.Dense(1) #Encoder output ie Decoder and Ecoder matrix multiplication and output single values

  def call(self,prev_dec_state,enc_states): # previous state of 256, new enc_states of 32
    """
    prev_dec_state is Decoder block before passing weights to next block is passed here ie dense layer
    """
    scores=self.w(tf.nn.tanh(
                  self.w_1(tf.expand_dims(prev_dec_state,-2)) +  #here (batch,32)--> (batch,1,32) as encoder output is (128,8,127)
                  self.w_2(enc_states))# (batch,8,32)
                  ) # from decoder shape is (batch,1,32) and encoder shape is (batch,8,32)

        #because tanh goes in self.w --> single outputs in scores as list and this will be passed into softmax to tell which
          #word in decoder has highest probability with encoder word
        # call fn will take embedding from Encoder, change dims to eg 32, same dims from Decoder,matmul, update weights and give single output as self.w.
        # we want self.w score should be equal to 1, so we use softmax below
        # self.w will be now passed again and again to subsequent words in Decoder
        # shape of self.w will be (batch,no of blocks, 1) 1 for single value of self.w

    # check shapes of below variables
    attention_weights=tf.nn.softmax(scores,axis=1) # word in output that corresponds to input, give max weightage to that word/scores
    #attention_weights shape is (batch,8,1),
    context_vector=attention_weights*enc_states  #now we again multiple by original enc_states(batch,8,32) ==> shape of (batch,8,32)
    print(context_vector.shape)
    context_vector=tf.reduce_sum(context_vector,axis=1) #(batch,8,32) --> vector of dims 256 and ready to be passed in decoder next block as single value
    print(context_vector.shape) # this will be passed from bottom in decoder
    return context_vector,attention_weights

In [53]:
bahdanau_attention=BahdanauAttention(256)
context_vector,attention_weights=bahdanau_attention(tf.zeros([128,32]),tf.zeros([128,8,32])) # prev_dec_state(256), enc_states(32)
print(context_vector.shape)
print(attention_weights.shape)

(128, 8, 32)
(128, 32)
(128, 32)
(128, 8, 1)


## Decoder

In [56]:
class Decoder(tf.keras.Model):
  def __init__(self,vocab_size,embedding_dim,dec_units,sequence_length):
    super(Decoder,self).__init__()
    self.embedding_dim=embedding_dim
    self.vocab_size=vocab_size
    self.dec_units=dec_units
    self.sequence_length=sequence_length

  def build(self,input_shape):
    self.dense=Dense(self.vocab_size,activation="softmax") # output from decoder block, ie output from top having same dims as vocab size
    self.gru=GRU(self.dec_units,return_sequences=True,return_state=True)
    self.attention=BahdanauAttention(self.dec_units) # as input vectors will enter into every single block
    self.embedding=Embedding(self.vocab_size,self.embedding_dim) # french words from bottom


  def call(self,x,hidden,shifted_target): #shifted_target are decoder blocks ie actual word getting passed into that block
    outputs=[]
    context_vectors=[]
    attention_weightss=[]
    shifted_target=self.embedding(shifted_target)  # create embedding for every block in output

    for t in range(0,self.sequence_length): # actual embedding layers
      context_vector,attention_weights=self.attention(hidden,x) # hidden is previous word encoding, x is input layer encoding ie initial 8 embedding dims(encoder output)
      dec_input=context_vector+shifted_target[:,t] # context_vector(x) is input encoding, shifted_target[:t] is only passing a specific word of that block, both are combined and passed from the bottom
      # encoder dims + actual word vector dims

      output,hidden=self.gru(tf.expand_dims(dec_input,1)) # gru takes 3D inputs, output of this block(from top--> encoder) is now saved, hidden passed from left
      #decoder input is now passed in gru
      outputs.append(output[:,0]) #output has shape->,batch_size,units

    outputs=tf.convert_to_tensor(outputs) #sequence_length,batch,units ie sequence_length will be auto added
    outputs=tf.transpose(outputs, perm=[1,0,2]) # we want batch_size,sequence_length,units so we do transpose

    outputs=self.dense(outputs) #vocab size, softmax
    return outputs,attention_weights

In [57]:
decoder=Decoder(VOCAB_SIZE,EMBEDDING_DIM,HIDDEN_UNITS,FRENCH_SEQUENCE_LENGTH)
outputs,attention_weights=decoder(encoder_output,tf.zeros([128,HIDDEN_UNITS]),tf.zeros([128,64]))
print(outputs.shape)
print(attention_weights.shape)

(128, 8, 127)
(128, 127)


InvalidArgumentError: ignored

In [ ]:
### ENCODER
input = Input(shape=(ENGLISH_SEQUENCE_LENGTH,), dtype="int64", name="input_1")
encoder=Encoder(VOCAB_SIZE,EMBEDDING_DIM,HIDDEN_UNITS)
encoder_output=encoder(input)

### DECODER
shifted_target=Input(shape=(FRENCH_SEQUENCE_LENGTH,), dtype="int64", name="input_2")
decoder=Decoder(VOCAB_SIZE,EMBEDDING_DIM,HIDDEN_UNITS,FRENCH_SEQUENCE_LENGTH)
decoder_output,attention_weightss=decoder(encoder_output,tf.zeros([1,HIDDEN_UNITS]),shifted_target) #tf.zeros([1,HIDDEN_UNITS]) is needed becuase previous hidden state doesn't exist in output ie [start]

### OUTPUT
bahdanau=Model([input,shifted_target],decoder_output)
bahdanau.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64)]         0           []                               
                                                                                                  
 encoder_8 (Encoder)            (None, 64, 256)      5645312     ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 64)]         0           []                               
                                                                                                  
 decoder_18 (Decoder)           ((None, 64, 20000),  10786593    ['encoder_8[0][0]',              
                                 (None, 64, 1))                   'input_2[0][0]']         

In [ ]:
class BLEU(tf.keras.metrics.Metric):
    def __init__(self,name='bleu_score'):
        super(BLEU,self).__init__()
        self.bleu_score=0

    def update_state(self,y_true,y_pred,sample_weight=None):
      y_pred=tf.argmax(y_pred,-1)
      self.bleu_score=0
      for i,j in zip(y_pred,y_true):
        tf.autograph.experimental.set_loop_options()

        total_words=tf.math.count_nonzero(i)
        total_matches=0
        for word in i:
          if word==0:
            break
          for q in range(len(j)):
            if j[q]==0:
              break
            if word==j[q]:
              total_matches+=1
              j=tf.boolean_mask(j,[False if y==q else True for y in range(len(j))])
              break

        self.bleu_score+=total_matches/total_words

    def result(self):
        return self.bleu_score/BATCH_SIZE

In [ ]:
bahdanau.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(5e-4),
    metrics=[BLEU()],
    run_eagerly=True)

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/nlp/translation/bahdanau_attention_1.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
history=bahdanau.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,)
    #callbacks=[model_checkpoint_callback])

Epoch 1/30
2812/2812 [==============================] - 790s 281ms/step - loss: 0.3764 - val_loss: 0.8106
Epoch 2/30
2812/2812 [==============================] - 784s 279ms/step - loss: 0.2681 - val_loss: 0.6852
Epoch 3/30
2812/2812 [==============================] - 781s 278ms/step - loss: 0.2046 - val_loss: 0.6187
Epoch 4/30
2812/2812 [==============================] - 781s 278ms/step - loss: 0.1673 - val_loss: 0.5875
Epoch 5/30
2812/2812 [==============================] - 779s 277ms/step - loss: 0.1427 - val_loss: 0.5829
Epoch 6/30
2812/2812 [==============================] - 779s 277ms/step - loss: 0.1251 - val_loss: 0.5645
Epoch 7/30
2812/2812 [==============================] - 777s 276ms/step - loss: 0.1119 - val_loss: 0.5586
Epoch 8/30
2812/2812 [==============================] - 777s 276ms/step - loss: 0.1016 - val_loss: 0.5745
Epoch 9/30
2812/2812 [==============================] - 780s 277ms/step - loss: 0.0932 - val_loss: 0.5652
Epoch 10/30
2812/2812 [=======================

KeyboardInterrupt: ignored

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('model_accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
bahdanau.evaluate(val_dataset)

274/274 [==============================] - 1865s 7s/step - loss: 0.7017 - bleu_1: 0.1561


[0.7016859650611877, 0.15608372877928317]

# Testing

In [ ]:
index_to_word={x:y for x, y in zip(range(len(french_vectorize_layer.get_vocabulary())),
                                   french_vectorize_layer.get_vocabulary())}

In [ ]:
def translator(english_sentence):
  tokenized_english_sentence=english_vectorize_layer([english_sentence])
  shifted_target='starttoken'

  for i in range(FRENCH_SEQUENCE_LENGTH):
    tokenized_shifted_target=french_vectorize_layer([shifted_target])
    output=bahdanau.predict([tokenized_english_sentence,tokenized_shifted_target])
    french_word_index=tf.argmax(output,axis=-1)[0][i].numpy()
    current_word=index_to_word[french_word_index]
    if current_word=='endtoken':
      break
    shifted_target+=' '+current_word
  return shifted_target[11:]

In [ ]:
translator('What makes you think that is not true?')

'questce qui te fait croire que ce nest pas vrai'

In [ ]:
translator('Have you ever watched a soccer under the rain?')

'astu déjà regardé un match de la pluie'

In [ ]:
translator('Great trees do not grow with ease, the stronger the winds, the stronger the trees')

'des arbres sont très [UNK] pas fort'

In [ ]:
translator('Everyone should water his or her tomato plants')

'tout le monde devrait manger sa propriété à la nage'

In [ ]:
word_to_index={y:x for x, y in zip(range(len(french_vectorize_layer.get_vocabulary())),
                                   french_vectorize_layer.get_vocabulary())}